In [1]:
import re
#import pyspark.sql.functions as F
from pyspark.sql import Row #Converte RDDs em objetos do tipo Row
from pyspark.sql.functions import col, isnan, when, count # Encontra a contagem para valores None, Null, Nan, etc.
from pyspark.sql.types import IntegerType, FloatType

from nltk.corpus import stopwords


# Módulos usados
from pyspark.streaming import StreamingContext
from pyspark import SparkContext
from requests_oauthlib import OAuth1Session
from operator import add
import requests_oauthlib
from time import gmtime, strftime
import requests
import time
import string
import ast
import json
#import re

In [2]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

# Create a local StreamingContext with two working thread and batch interval of 1 second
ssc = StreamingContext(sc, 1)

# Create a DStream that will connect to hostname:port, like localhost:9999
lines = ssc.socketTextStream("127.0.0.1", 5554)

In [3]:
# Cria uma lista vazia para os resultados
resultados = []

# Essa função classifica os tweets, aplicando as features do modelo criado anteriormente
def classifica_tweet(tweet):
    return(tweet, 'ok')


# Essa função retorna o texto do Twitter
def get_tweet_text(rdd):
    for line in rdd:
        tweet = line.strip()
        translator = str.maketrans({key: None for key in string.punctuation})
        tweet = tweet.translate(translator)
        tweet = tweet.split(' ')
        tweet_lower = []
        for word in tweet:
            tweet_lower.append(word.lower())
    return(classifica_tweet(tweet_lower))


# Essa função salva o resultado dos batches de Tweets junto com o timestamp
def output_rdd(rdd):
    global resultados
    pairs = rdd.map(lambda x: (get_tweet_text(x)[1],1))
    counts = pairs.reduceByKey(add)
    output = []
    for count in counts.collect():
        output.append(count)
    result = [time.strftime("%I:%M:%S"), output]
    resultados.append(result)
    print(result)
    
    

In [3]:
def saida(rdd):
    global resultados
    # Count each word in each batch
    #pairs = words.map(lambda word: (word, 1))
    #wordCounts = pairs.reduceByKey(lambda x, y: x + y)

    #output = []
    #for count in wordCounts.collect():
    #    word = limparTexto(count[0])
    #    if word != '':
    #        if count[0] not in stop_words:
    #            output.append(count)

    #resultados = []
    #result = [time.strftime("%I:%M:%S"), output]
    resultados.append(rdd)
    print(resultados)
    
    return(rdd)


In [ ]:
por = stopwords.words('portuguese')
eng = stopwords.words('english')
spa = stopwords.words('spanish')

stop_words = por + eng + spa

In [3]:
resultados = []
#words_trans = lines.map(lambda x: limparTexto(x)).map(lambda x: removerStopwords(x))

# Split each line into words
words = lines.flatMap(lambda line: line.split(" "))

# Count each word in each batch
pairs = words.map(lambda word: (word, 1))
wordCounts = pairs.reduceByKey(lambda x, y: x + y)


#resultados.append(wordCounts)

#coord_stream = lines.map(lambda line: ast.literal_eval(line))
# A função foreachRDD() aplica uma função a cada RDD to streaming de dados
#coord_stream.foreachRDD(lambda t, rdd: output_rdd(rdd))

#wordCounts.foreachRDD(lambda t, rdd: output_rdd(rdd))




# Print the first ten elements of each RDD generated in this DStream to the console
wordCounts.pprint()

In [4]:
ssc.start()             # Start the computation
ssc.awaitTerminationOrTimeout(120)  # Wait for the computation to terminate
ssc.stop()

-------------------------------------------
Time: 2022-10-10 11:32:29
-------------------------------------------

-------------------------------------------
Time: 2022-10-10 11:32:30
-------------------------------------------

-------------------------------------------
Time: 2022-10-10 11:32:31
-------------------------------------------

-------------------------------------------
Time: 2022-10-10 11:32:32
-------------------------------------------

-------------------------------------------
Time: 2022-10-10 11:32:33
-------------------------------------------

-------------------------------------------
Time: 2022-10-10 11:32:34
-------------------------------------------

-------------------------------------------
Time: 2022-10-10 11:32:35
-------------------------------------------

-------------------------------------------
Time: 2022-10-10 11:32:36
-------------------------------------------

-------------------------------------------
Time: 2022-10-10 11:32:37
----------

In [13]:
teste = wordCounts.map(lambda line: ast.literal_eval(line))
type(teste)

pyspark.streaming.dstream.TransformedDStream

In [18]:
# Essa função classifica os tweets, aplicando as features do modelo criado anteriormente
def classifica_tweet(tweet):
    return(tweet, 'ok')

# Essa função retorna o texto do Twitter
def get_tweet_text(rdd):
    for line in rdd:
        tweet = line.strip()
        tweet_lower = []
        for word in tweet:
            tweet_lower.append(word.lower())
    return(classifica_tweet(tweet_lower))

# Essa função salva o resultado dos batches de Tweets junto com o timestamp
def output_rdd(rdd):
    global resultados
    pairs = rdd.map(lambda x: (get_tweet_text(x)[0],1))
    counts = pairs.reduceByKey(add)
    output = []
    for count in counts.collect():
        output.append(count)
    result = [time.strftime("%I:%M:%S"), output]
    resultados.append(result)
    print(result)

In [19]:
# A função foreachRDD() aplica uma função a cada RDD to streaming de dados
wordCounts.foreachRDD(lambda t, rdd: output_rdd(rdd))

Py4JJavaError: An error occurred while calling z:org.apache.spark.streaming.api.python.PythonDStream.callForeachRDD.
: java.lang.IllegalStateException: Adding new inputs, transformations, and output operations after stopping a context is not supported
	at org.apache.spark.streaming.dstream.DStream.validateAtInit(DStream.scala:230)
	at org.apache.spark.streaming.dstream.DStream.<init>(DStream.scala:67)
	at org.apache.spark.streaming.dstream.ForEachDStream.<init>(ForEachDStream.scala:39)
	at org.apache.spark.streaming.dstream.DStream.foreachRDD(DStream.scala:655)
	at org.apache.spark.streaming.dstream.DStream.$anonfun$foreachRDD$3(DStream.scala:640)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.SparkContext.withScope(SparkContext.scala:763)
	at org.apache.spark.stre

In [23]:
def removerStopwords(texto):
    por = stopwords.words('portuguese')
    eng = stopwords.words('english')
    spa = stopwords.words('spanish')

    stop_words = por + eng + spa
    
    texto = ' '.join(palavra for palavra in texto.split(' ') if palavra not in stop_words)
    return texto

def limparTexto(texto):
    # Remove emojis
    emoji_pattern = re.compile("["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "]+", flags=re.UNICODE)
    texto = emoji_pattern.sub(r'', texto) # no emoji
    
    # Remove non-english chars: chinese, arabic, korean, etc...
    nonenglish_pattern = re.compile(u'[^\u0000-\u05C0\u2100-\u214F]+', flags=re.UNICODE)
    texto = nonenglish_pattern.sub(r'', texto)
    
    #Vamos transformar o texto em lowercase, remover textos entre colchetes, links, pontuações e palavras que contenham números.
    texto = str(texto).lower()
    texto = re.sub('\[.*?\]', '', texto) #Removendo textos entre colchetes
    texto = re.sub('<.*?>+', '', texto)  # Remove textos entre <>
    texto = re.sub('https?://\S+|www\.\S+', '', texto) #Removendo links
    texto = re.sub('[@#]\S+', '', texto) #Removendo arrobas e hashtags
    texto = re.sub('\w*\d\w*', '', texto) #Remove palavras contendo dígitos no meio.
        
    texto = re.sub(r'[%@#\t\n\r]+', '', texto) #Remove caracteres especiais
    texto = re.sub(r'[ ]+', ' ', texto) #Remove mais do que um espaço em branco
    
    return (texto)

---

---

---

---

---

Links: 

https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#programming-model

https://github.com/syalanuj/youtube/blob/main/spark_streaming_with_python_in_12_minutes/spark_st_run.ipynb

https://github.com/Krupique/cursos-datascience-conteudo/blob/main/DSA_DS-02-BigData%20Analytics%20com%20Python%20e%20Spark/05%20-%20Introducao-SparkStreaming.ipynb


In [15]:
lista = [
    'É necessário muita ação',
    'I plan nothing on my weekend ！#cute #可爱 #萌 #cat https://t.co/JNbftLiVOM',
         '@PAVGOD: Whenever someone asks if Pavlov is a cuddle dog, \
         I try to explain that this is his reaction 😂 https://t.co/KGrI4pHEXV',
         'Cat feed cat https://t.co/1Avv8qhT4J 🗣️"Tenim el deure de complir amb el mandat del 52 %  \
         davançar cap a la independència'
        ]

rdd = sc.parallelize(lista)

rdd.take(5)

['É necessário muita ação',
 'I plan nothing on my weekend ！#cute #可爱 #萌 #cat https://t.co/JNbftLiVOM',
 '@PAVGOD: Whenever someone asks if Pavlov is a cuddle dog,          I try to explain that this is his reaction 😂 https://t.co/KGrI4pHEXV',
 'Cat feed cat https://t.co/1Avv8qhT4J 🗣️"Tenim el deure de complir amb el mandat del 52 %           davançar cap a la independència']

In [ ]:
# Essa função salva o resultado dos batches de Tweets junto com o timestamp
def output_rdd(rdd):
    global resultados
    pairs = rdd.map(lambda x: (get_tweet_text(x)[1],1))
    counts = pairs.reduceByKey(add)
    output = []
    for count in counts.collect():
        output.append(count)
    result = [time.strftime("%I:%M:%S"), output]
    resultados.append(result)
    print(result)
    

In [31]:
rdd1 = rdd.flatMap(lambda line: line.split(' '))

filt = rdd1.filter(lambda line: removerStopwords(line))

pairs = rdd1.map(lambda x: (x.lower(), 1))

counts = pairs.reduceByKey(add)

In [39]:
counts.take(20)

[('i', 2),
 ('#可爱', 1),
 ('', 19),
 ('😂', 1),
 ('https://t.co/kgri4phexv', 1),
 ('cat', 2),
 ('feed', 1),
 ('el', 2),
 ('deure', 1),
 ('cap', 1),
 ('nothing', 1),
 ('weekend', 1),
 ('！#cute', 1),
 ('@pavgod:', 1),
 ('whenever', 1),
 ('someone', 1),
 ('a', 2),
 ('dog,', 1),
 ('to', 1),
 ('that', 1)]

In [60]:
por = stopwords.words('portuguese')
eng = stopwords.words('english')
spa = stopwords.words('spanish')

stop_words = por + eng + spa

output = []
for count in counts.collect():
    
    word = limparTexto(count[0])
    if word != '':
        if count[0] not in stop_words:
            output.append(count)

In [61]:
resultados = []
result = [time.strftime("%I:%M:%S"), output]
resultados.append(result)
print(result)

['10:47:51', [('cat', 2), ('feed', 1), ('deure', 1), ('cap', 1), ('nothing', 1), ('weekend', 1), ('whenever', 1), ('someone', 1), ('dog,', 1), ('reaction', 1), ('amb', 1), ('davançar', 1), ('plan', 1), ('pavlov', 1), ('try', 1), ('🗣️"tenim', 1), ('complir', 1), ('independència', 1), ('necessário', 1), ('muita', 1), ('ação', 1), ('asks', 1), ('cuddle', 1), ('explain', 1), ('mandat', 1)]]


In [ ]:
rdd3 = rdd.map(lambda x: limparTexto(x)).map(lambda x: removerStopwords(x))

rdd3.take(20)

In [ ]:
rdd.take(5)

## Project

In [1]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc
from collections import namedtuple

# Can only run this once. restart your kernel for any errors.

ssc = StreamingContext(sc, 10 )
sqlContext = SQLContext(sc)
socket_stream = ssc.socketTextStream("127.0.0.1", 5554)

lines = socket_stream.window( 100 )

fields = ("text")

# Use Parenthesis for multiple lines or use \.
Tweet = namedtuple( 'Tweet', fields )
( lines.flatMap( lambda text: text.split( " " ) ) 
     #.filter( lambda word: word.lower().startswith("http") )  
     .map( lambda word: ( word.lower(), 1 ) ) 
     .reduceByKey( lambda a, b: a + b ) 
     .map( lambda rec: Tweet( rec[0], rec[1] ) ) 
     .foreachRDD( lambda rdd: rdd.toDF().sort( desc("count") ) 
     .limit(100).registerTempTable("tweets") ) ) # Registers to a table.

## Now run TweetListener.py

## Spark

In [2]:
ssc.start()    

In [ ]:
import time
from IPython import display
import matplotlib.pyplot as plt
import seaborn as sns
# Only works for Jupyter Notebooks!
%matplotlib inline 

In [ ]:
help(Tweet)

In [ ]:
Tweet.text

In [ ]:
tweets = sqlContext.sql( 'Select * from tweets' )
df = tweets.toPandas()

df.head(15)

In [ ]:
ssc.stop()  